In [1]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
import numpy as np

## Func

In [2]:
## 데이터 구성함수
def multivariate_data(dataset, target, start_index, end_index, history_size,
                      target_size, step, single_step=False):
    data = []
    labels = []

    start_index = start_index + history_size
    if end_index is None:
        end_index = len(dataset) - target_size

    for i in range(start_index, end_index):
        indices = range(i-history_size, i, step)
        data.append(dataset[indices])

        if single_step:
            labels.append(target[i+target_size])
        else:
            labels.append(target[i:i+target_size])

    return np.array(data), np.array(labels)

## Data

In [3]:
df_ori = pd.read_csv('data/data_featsel.csv', index_col=0)
df_ori['spread'] = df_ori['spread'] - df_ori['spread'].shift(1)

In [4]:
## dataset = df_ori[['spread_ratio', 'spread_ratio', 'vwap_near_rets_abs', 'duetime', 'volume_near_log', 'spread']].dropna()

In [5]:
dataset = df_ori[['spread_ratio', 'spread_ratio', 'vwap_near_rets_abs', 'duetime', 'volume_near_log', 'spread']][df_ori['epi_no'] > 1]

## Restructure

In [6]:
df = dataset.copy()
dataset = df.values
train_start = 0
train_end = len(df_ori[df_ori['epi_no'] <= 5]) ## 5개 분기
history_size = 8
target_size = 0
step = 1
single_step = True

valid_start = train_end-history_size
valid_end = len(df_ori[df_ori['epi_no'] <= 8]) ## 3개 분기

test_start = valid_end-history_size
test_end = None

target = dataset[:, -1]

X_train, y_train = multivariate_data(dataset, target, train_start, train_end, history_size, target_size, step, single_step)
X_valid, y_valid = multivariate_data(dataset, target, valid_start, valid_end, history_size, target_size, step, single_step)
X_test, y_test = multivariate_data(dataset, target, test_start, test_end, history_size, target_size, step, single_step)

train_date = pd.DataFrame(df[train_start+history_size:train_end].index)
valid_date = pd.DataFrame(df[valid_start+history_size:valid_end].index)
test_date = pd.DataFrame(df[test_start+history_size:test_end].index)

In [7]:
np.save(f'data/RNN/train_X.npy',X_train)
np.save(f'data/RNN/train_y.npy',y_train)
np.save(f'data/RNN/valid_X.npy',X_valid)
np.save(f'data/RNN/valid_y.npy',y_valid)
np.save(f'data/RNN/test_X.npy',X_test)
np.save(f'data/RNN/test_y.npy',y_test)

train_date.to_csv(f'data/RNN/train_ans.csv')
valid_date.to_csv(f'data/RNN/valid_ans.csv')
test_date.to_csv(f'data/RNN/test_ans.csv')